In [2]:
import kagglehub
import pandas as pd
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import chromadb
import time
from langchain.vectorstores import Chroma

load_dotenv(dotenv_path="Kuan.env") 
openai_key = os.getenv("OPENAI_API_KEY")


# Download dataset (returns the folder path)
dataset_dir = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset")

# List files in the folder to find the CSV
files = os.listdir(dataset_dir)
print("Downloaded files:", files)

# Assuming the file you want is the only one or named clearly:
csv_path = os.path.join(dataset_dir, files[0])  # Adjust if multiple files

# Read the file (check whether it's tab-separated first!)
df = pd.read_csv(csv_path, sep=',', on_bad_lines='skip')
df_json = df.to_dict(orient='records')

documents = [
    Document(
        page_content="; ".join([f"{k}: {str(v)}" for k, v in record.items()]),
        metadata=record
    )
    for record in df_json
]

embedding_model = OpenAIEmbeddings()


Downloaded files: ['amazon.csv']


C:\Users\Kuan-ChihLee\AppData\Local\Temp\ipykernel_28744\3784915400.py:38: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


In [ ]:
embedding_model = OpenAIEmbeddings()

persist_dir = "./vector_database"
collection_name = "my_collection"

def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

batch_size = 50  # smaller batch to avoid token limit

for batch_idx, batch_docs in enumerate(chunk_list(documents, batch_size)):
    vectorstore = Chroma.from_documents(
        documents=batch_docs,
        embedding=embedding_model,
        collection_name=f"{collection_name}_batch_{batch_idx}",
        persist_directory=persist_dir,
    )
    vectorstore.persist()
    print(f"Persisted batch {batch_idx + 1}")

print("All batches processed!")


In [3]:

RETRIEVED_DOCS_N = 5  

retriever = Chroma(
    collection_name="my_combined_collection",  
    embedding_function=embedding_model,        
    persist_directory="./vector_database"
).as_retriever(search_kwargs={"k": RETRIEVED_DOCS_N})


C:\Users\Kuan-ChihLee\AppData\Local\Temp\ipykernel_3916\1984886844.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  retriever = Chroma(


In [ ]:
query = "what the most Lightning Fast USB "
results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results):
    print(f"\n--- Document {i+1} ---\n{doc.page_content[:300]}...")


C:\Users\Kuan-ChihLee\AppData\Local\Temp\ipykernel_3916\1349448061.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



--- Document 1 ---
product_id: B07XLCFSSN; product_name: Amazonbasics Nylon Braided Usb-C To Lightning Cable, Fast Charging Mfi Certified Smartphone, Iphone Charger (6-Foot, Dark Grey); category: Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables; discounted_price: ₹899; actual_price: ₹1...

--- Document 2 ---
product_id: B09W5XR9RT; product_name: Duracell USB C To Lightning Apple Certified (Mfi) Braided Sync & Charge Cable For Iphone, Ipad And Ipod. Fast Charging Lightning Cable, 3.9 Feet (1.2M) - Black; category: Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables; discount...

--- Document 3 ---
product_id: B09C6HXFC1; product_name: Duracell USB Lightning Apple Certified (Mfi) Braided Sync & Charge Cable For Iphone, Ipad And Ipod. Fast Charging Lightning Cable, 3.9 Feet (1.2M) - Black; category: Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables; discounted_pr...

--- Document 4 ---
product_